<a href="https://colab.research.google.com/github/chavalvit-k/Admissions-Prediction/blob/main/Admissions_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Libraries

In [50]:
!pip install scikeras

In [62]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam

from scikeras.wrappers import KerasRegressor

# Import Dataset

In [2]:
df = pd.read_csv("admissions_data.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


# Explore Dataset

In [3]:
df.drop(["Serial No."], axis=1).describe()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


# Split Training Set and Testing Set

In [4]:
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]

In [5]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [6]:
y.head()

,Chance of Admit
0,0.92
1,0.76
2,0.72
3,0.80
4,0.65


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 42)

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build Regression Model

In [86]:
def build_model(neurons = 8):
  model = Sequential([
    InputLayer(shape = (X_train_scaled.shape[1],)),
    Dense(neurons, activation = "relu"),
    Dense(1)
  ])

  model.compile(optimizer = Adam(learning_rate = 0.01), loss = "mse", metrics = ["mae"])

  return model

In [120]:
def build_best_model():
  model = KerasRegressor(model=build_model, verbose = 0)

  param_grid = {
    "batch_size": [1, 4],
    "epochs": [10, 20, 30],
    "model__neurons": [4, 8]
  }

  grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = make_scorer(mean_squared_error, greater_is_better = False), return_train_score = True, cv = 3)
  grid_result = grid.fit(X_train_scaled, y_train, verbose = 0)

  print("Best score:", grid_result.best_score_)
  print("Best result:", grid_result.best_params_)

  best_params = grid_result.best_params_
  best_model = build_model(neurons = best_params["model__neurons"])

  best_model.fit(X_train_scaled, y_train, batch_size = best_params["batch_size"], epochs = best_params["epochs"], verbose=1)

  return best_model

In [121]:
best_model = build_best_model()
best_model.summary()

Best score: -0.004388642463013102
Best result: {'batch_size': 1, 'epochs': 20, 'model__neurons': 8}
Epoch 1/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3727 - mae: 0.4060
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0073 - mae: 0.0676
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0050 - mae: 0.0556
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0046 - mae: 0.0519
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0055 - mae: 0.0563
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0043 - mae: 0.0477
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0046 - mae: 0.0517
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0038 - mae: 0.0462
Epoch 9/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0047 - mae: 0.0520
Epoch 10/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0055 - mae: 0.0547
Epoch 11/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0051 - mae: 0.0548
Epoch 1

Model: "sequential_342"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_684 (Dense)                    │ (None, 8)                   │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_685 (Dense)                    │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 221 (888.00 B)

 Trainable params: 73 (292.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 148 (596.00 B)

# Evaluate Model

In [122]:
mse, mae = best_model.evaluate(X_test_scaled, y_test)
print("MSE: ", mse)
print("MAE: ", mae)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0050 - mae: 0.0527  
MSE:  0.004604737740010023
MAE:  0.05091604217886925


In [125]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train_scaled, y_train)
y_pred = dummy_regr.predict(X_test_scaled)
baseline_MAE = mean_absolute_error(y_test, y_pred)
print("Baseline MAE:", baseline_MAE)

Baseline MAE: 0.116268


In [127]:
y_pred = best_model.predict(X_test_scaled)
r_square = r2_score(y_test, y_pred)
print("R-Square:", r_square)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
R-Square: 0.7748294486105487
